# math

> Basic math routine

In [ ]:
#| default_exp cli/math

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import zarr
import numpy as np
from decorrelation.cli.utils.logging import get_logger

In [ ]:
#| export
import logging
import zarr
import numpy as np
import numexpr as ne

import dask
from dask import array as da
from dask.distributed import Client, LocalCluster, progress

from decorrelation.cli.utils.logging import log_args, de_logger

In [ ]:
#| export
def _math(operation=str,
          **data):
    return ne.evaluate(operation,data)

In [ ]:
#| export
@log_args
@de_logger
def de_math(output:str, # path to output
            operation:str, # operation
            **data):
    '''
    Basic math manipulation. Only elementwise operations are supported. Only one output is supported.
    '''
    output_path = output
    logger = logging.getLogger(__name__)
    for name, path in data.items():
        path_zarr = zarr.open(path,mode='r'); logger.zarr_info(name,path_zarr)
    logger.info('starting dask local cluster.')
    with LocalCluster() as cluster, Client(cluster) as client:
        logger.info('dask local cluster started.')
        names = []; darrs = []
        
        for name, path in data.items():
            names.append(name)
            darr = da.from_zarr(path); logger.darr_info(name,darr)
            darrs.append(darr)
        darr0 = darrs[0]
        darrs = [darr.to_delayed() for darr in darrs]

        output_delayed = np.empty_like(darrs[0],dtype=object)
        math_delayed = dask.delayed(_math,pure=True,nout=1)
        with np.nditer(darrs[0],flags=['multi_index','refs_ok'], op_flags=['readwrite']) as it:
            for block in it:
                idx = it.multi_index
                math_kw = {}
                for name, darr in zip(names, darrs):
                    math_kw[name] = darr[idx]
                output_delayed[idx] = math_delayed(operation,**math_kw)
                output_delayed[idx] = da.from_delayed(output_delayed[idx],shape=darr0.blocks[idx].shape,meta=np.array(()))
        output = da.block(output_delayed.tolist())
        _output = output.to_zarr(output_path,overwrite=True,compute=False)
        logger.info('computing graph setted. doing all the computing.')
        futures = client.persist(_output)
        progress(futures,notebook=False)
        da.compute(futures)
        logger.info('computing finished.')
    logger.info('dask cluster closed.')

This function is based on [Numexpr](https://numexpr.readthedocs.io/en/latest/index.html).
All [operators](https://numexpr.readthedocs.io/en/latest/user_guide.html#supported-operators) and 
[functions](https://numexpr.readthedocs.io/en/latest/user_guide.html#supported-functions)
supported in`Numexpr` are supported except reduction operations.

Usage:

In [ ]:
a = np.random.rand(100,100).astype(np.float32)
b = np.random.rand(100,100).astype(np.float32)
a_zarr = zarr.open('./math/a.zarr','w',shape=a.shape,dtype=a.dtype,chunks=(10,10))
b_zarr = zarr.open('./math/b.zarr','w',shape=b.shape,dtype=b.dtype,chunks=(10,10))
a_zarr[:] = a; b_zarr[:] = b

In [ ]:
logger = get_logger()

In [ ]:
de_math('./math/c.zarr','sin(a)*exp(b)/2',a='./math/a.zarr',b = './math/b.zarr')

2023-11-01 18:01:47 - logging_args - INFO - fetching args:
2023-11-01 18:01:47 - logging_args - INFO - output = './math/c.zarr'
2023-11-01 18:01:47 - logging_args - INFO - operation = 'sin(a)*exp(b)/2'
2023-11-01 18:01:47 - logging_args - INFO - data = {'a': './math/a.zarr', 'b': './math/b.zarr'}
2023-11-01 18:01:47 - logging_args - INFO - fetching args done.
2023-11-01 18:01:47 - zarr_info - INFO - a zarray shape: (100, 100)
2023-11-01 18:01:47 - zarr_info - INFO - a zarray chunks: (10, 10)
2023-11-01 18:01:47 - zarr_info - INFO - a zarray dtype: float32
2023-11-01 18:01:47 - zarr_info - INFO - b zarray shape: (100, 100)
2023-11-01 18:01:47 - zarr_info - INFO - b zarray chunks: (10, 10)
2023-11-01 18:01:47 - zarr_info - INFO - b zarray dtype: float32
2023-11-01 18:01:47 - de_math - INFO - starting dask local cluster.
2023-11-01 18:01:52 - de_math - INFO - dask local cluster started.
2023-11-01 18:01:52 - darr_info - INFO - a dask array shape: (100, 100)
2023-11-01 18:01:52 - darr_info

In [ ]:
c = zarr.open('./math/c.zarr','r')[:]
np.testing.assert_array_almost_equal(c,np.sin(a)*np.exp(b)/2)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()